This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [185]:
# First, import the relevant modules
import requests as req
import collections as coll
import statistics as stat

In [216]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url_api = ('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' + API_KEY)
r = req.get(url_api + '&start_date=2019-04-24&end_date=2019-04-24')
carl = r.json()


In [217]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(carl)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2019-04-24', 'end_date': '2019-04-24', 'frequency': 'daily', 'data': [['2019-04-24', None, 84.4, 83.0, 83.5, None, 220545.0, 18418288.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Create 2017 Stock Prices Dictionary

In [97]:
# get 2017 data
r = req.get(url_api + '&start_date=2017-01-01&end_date=2018-01-01')
carl = r.json()
print('Column Names: ' + str(carl['dataset_data']['column_names']) + '\n')
print('Example Day:', carl['dataset_data']['data'][0])

Column Names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']

Example Day: ['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [94]:
# get column names and list of values to create dictionary
columns = carl['dataset_data']['column_names']
daily_values = carl['dataset_data']['data']

In [95]:
# create dictionary using Collections library
carl_dict = coll.OrderedDict()
for day in daily_values:
    carl_dict[day[0]] = dict(zip(columns[1:], day[1:]))        

In [119]:
print('carl_dict Sample:\n\n', next(iter(carl_dict.items())))

carl_dict Sample:

 ('2017-12-29', {'Open': 51.76, 'High': 51.94, 'Low': 51.45, 'Close': 51.76, 'Change': None, 'Traded Volume': 34640.0, 'Turnover': 1792304.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None})


#### Highest and Lowest Stock Prices

In [104]:
# Create a Dictionary for High and Low Values
highs = coll.OrderedDict()
lows = coll.OrderedDict()
for date in carl_dict:
    highs[date] = carl_dict[date]['High']
    lows[date] = carl_dict[date]['Low']
    
# Get Highest and Lowest Stock Date Price
# Highest
highest = max(highs, key=highs.get)
print('Highest:', highest, highs[maximum])
# Lowest
lowest = min(lows, key=lows.get)
print('Lowest:\t', lowest, lows[lowest])

Highest: 2017-12-14 53.54
Lowest:	 2017-01-23 33.62


#### Largest Single Day Price Change (High - Low) 

In [168]:
# create a change dictionary
change = coll.OrderedDict()
for date in carl_dict:
    change[date] = (carl_dict[date]['High'] - carl_dict[date]['Low'])
# Biggest Change
largest = max(change, key=change.get)
print('Largest High-Low Change:', largest, change[largest])

Biggest Change: 2017-05-11 2.8100000000000023


#### Largest Two Day Closing Price Change

In [175]:
# create a closing price dictionary
closing = coll.OrderedDict()
for date in carl_dict:
    closing[date] = carl_dict[date]['Close']
    
# make copy to remove the first entry of closing
# we will use this to later compare two consecutive dates
no_dec29 = closing.copy()
del no_dec29['2017-12-29']

# create final two_day dict containing compared dates
# and closing price difference 
two_day = coll.OrderedDict()
for (date1, close1), (date2, close2) in zip(closing.items(), no_dec29.items()):
    two_day[date1 + " : " + date2] = abs(close1 - close2)
# print largest closing day price difference observed
largest1 = max(two_day, key=two_day.get)
print('Dates Compared: ', largest1, '\nDifference:\t',two_day[largest1])

Dates Compared:  2017-08-09 : 2017-08-08 
Difference:	 2.559999999999995


#### 2017 Average Daily Trading Volume 

In [199]:
# Create list of daily traded volumes
trade = []
for date in carl_dict:
    trade.append(carl_dict[date]['Traded Volume'])
# find average of daily traded volume list 
# print it
print('Average Daily Trading Volume: ', sum(trade)/len(trade))

Average Daily Trading Volume:  89124.33725490196


#### 2017 Median Daily Trading Volume

In [213]:
# Check Number of Days Recorded in 2017 Stock Price
import math 
# since len(trade) = 255 is odd 
# we just pick the middle number of a sorted list
print('Median Trading Volume: ', sorted(trade)[math.trunc(len(trade)/2)+1])

Median Trading Volume:  76600.0
